## 1. 시놉시스 전처리

**설명:**

1.  **특수문자 제거:**
    *   `re.sub(r"[^가-힣A-Za-z0-9.,\"']+", " ", synopsis)`: 정규 표현식을 사용하여 한글, 영문자, 숫자, 마침표(.), 콤마(,), 쌍따옴표("), 작은따옴표('), 공백을 제외한 모든 문자를 공백으로 대체합니다.
2.  **불용어 제거:**
    *   `stopwords`: 한국어 불용어 리스트를 직접 정의합니다. (조사, 어미, 접속사 등)
    *   `filtered_words = [word for word in words if word not in stopwords]`: 리스트 컴프리헨션을 사용하여 불용어가 아닌 단어만 선택합니다.
3.  **어간 추출/원형 복원 (Stemming/Lemmatization):**
    *   이 부분은 선택 사항입니다. 한국어의 경우 KoNLPy와 같은 라이브러리를 사용하여 더 정교한 어간 추출/원형 복원을 수행할 수 있습니다. 여기서는 간단하게 조사, 어미 제거 수준으로 대체했습니다.
4. **소문자화**
    *   `lower()`: 영어가 있을 경우를 대비하여 소문자로 변환합니다.

In [ ]:
import re
from collections import Counter

def preprocess_synopsis(synopsis):
    """
    영화 시놉시스 텍스트를 전처리하는 함수.

    Args:
        synopsis: 전처리할 시놉시스 문자열.

    Returns:
        전처리된 시놉시스 문자열.
    """

    # 1. 특수문자 제거 (., ", ' 제외)
    synopsis = re.sub(r"[^가-힣A-Za-z0-9.,\"']+", " ", synopsis) # 한글, 영어, 숫자, .,",', 공백을 제외하고 모두 제거

    # 2. 불용어 제거 (조사, 어미 등) - 직접 정의
    stopwords = ["은", "는", "이", "가", "을", "를", "에게", "의", "와", "과", "만", "도", "에", "고", "다", "것", "로", "게"]  # 불용어 리스트

    words = synopsis.split()
    filtered_words = [word for word in words if word not in stopwords] # 불용어가 아닌 단어만 선택

    # 3. 어간 추출/원형 복원 (선택적) - stemming/lemmatization. 여기서는 간단하게 조사, 어미 제거로 대체.

    # 4. 소문자 변환 (영어)
    preprocessed_synopsis = " ".join(filtered_words).lower()  # 단어들을 다시 문장으로 합치기

    return preprocessed_synopsis

# 시놉시스 예시
synopsis = """
"나한테 별로 고마워하지 않아도 돼요" 까칠한 어른 윤서
"한 번 쯤은 자기를 믿어주는 사람이 있으면 좋잖아요" 꿈 없는 청년 수찬

시청 정기간행물의 인터뷰어 '윤서'에게 사람의 온기는 한여름의 습하고 불쾌한 더위 같은 것.
그러던 어느 날, 청년 배달원 '수찬'과 실랑이를 벌이고 만다.
이후 인터뷰 자리에서 우연찮게 다시 만나게 되는데...

윤서와 수찬, 두 사람의 불편한 만남은 조금씩 서로를 건드린다.
"""

# 전처리
preprocessed_synopsis = preprocess_synopsis(synopsis)
print("전처리된 시놉시스:\n", preprocessed_synopsis)


## 2. 토큰 추출

**설명:**

*   `text.split()`: 가장 간단한 토큰화 방법인 공백 기준 토큰화를 수행합니다.

In [ ]:
def tokenize(text):
    """
    텍스트를 토큰화하는 함수.

    Args:
        text: 토큰화할 텍스트 문자열.

    Returns:
        토큰 리스트.
    """
    tokens = text.split()  # 공백 기준으로 토큰화 (가장 간단한 방법)
    return tokens

# 전처리된 시놉시스로 토큰 추출
tokens = tokenize(preprocessed_synopsis)
print("토큰:\n", tokens)

## 3. TF-IDF 벡터화

**설명:**

1.  **`calculate_tf` 함수:**
    *   `Counter(tokens)`:  `collections.Counter`를 사용하여 토큰 리스트에서 각 토큰의 빈도를 계산합니다.
2.  **`calculate_idf` 함수:**
    *   `df = defaultdict(int)`: 각 단어의 DF(Document Frequency, 단어가 나타난 문서의 수)를 저장합니다.
    *   중첩 반복문과 `set(tokens)`: 각 문서의 토큰 목록을 순회하면서, 중복을 제거한 토큰 집합을 만들고, `df` 값을 증가시킵니다.
    *    `idf[token] = np.log(num_docs / (1 + freq))`: IDF를 계산합니다.
3.  **`calculate_tfidf` 함수:**
    *   TF 값과 IDF 값을 곱하여 TF-IDF 값을 계산합니다.
4.  **`vectorize_tfidf`함수:**
    *   `word_index`: 전체 어휘(단어 집합)에 있는 각 단어에 고유한 인덱스를 할당합니다. 이렇게 해야 벡터의 각 차원이 어떤 단어에 해당하는지 알 수 있습니다.  `word_index`는 모든 문서에서 추출한 전체 단어 집합을 기반으로 만들어야 합니다.
    *   `vector = np.zeros(len(word_index))`: 0으로 초기화된 벡터를 만듭니다. 이 벡터의 크기는 전체 어휘의 크기와 같습니다.
    *   반복문을 통해 `tfidf` 딕셔너리에 있는 각 단어의 TF-IDF 값을 해당 단어의 인덱스에 맞게 벡터에 할당합니다.

In [ ]:
import numpy as np
from collections import defaultdict

def calculate_tf(tokens):
    """
    토큰 리스트에 대한 TF 값을 계산하는 함수.

    Args:
        tokens: 토큰 리스트.

    Returns:
        TF 딕셔너리 (key: 토큰, value: TF 값).
    """
    tf = Counter(tokens)  # Counter를 사용하여 각 토큰의 빈도 계산
    return tf

def calculate_idf(documents_tokens):
    """
    여러 문서의 토큰 리스트에 대한 IDF 값을 계산하는 함수.

    Args:
        documents_tokens: 문서별 토큰 리스트의 리스트 (예: [["나", "영화"], ["영화", "재미"]])

    Returns:
        IDF 딕셔너리 (key: 토큰, value: IDF 값).
    """

    num_docs = len(documents_tokens) # 문서의 총 개수
    df = defaultdict(int)  # DF (Document Frequency)

    for tokens in documents_tokens: # 문서별 토큰 리스트를 순회
        for token in set(tokens): # 중복을 제거하고 토큰 순회
            df[token] += 1       # 토큰이 나타난 문서 개수 카운트

    idf = {}
    for token, freq in df.items():   # 모든 토큰에 대한 IDF 값 계산
        idf[token] = np.log(num_docs / (1 + freq)) # IDF값 계산

    return idf

def calculate_tfidf(tf, idf):
    """
    TF와 IDF를 곱하여 TF-IDF 값을 계산하는 함수.

    Args:
        tf: TF 딕셔너리.
        idf: IDF 딕셔너리.

    Returns:
        TF-IDF 딕셔너리 (key: 토큰, value: TF-IDF 값).
    """
    tfidf = {}
    for token, tf_value in tf.items(): # 모든 토큰에 대한 TF-IDF 값을 계산합니다
        if token in idf:              # 해당 토큰이 IDF 딕셔너리에 있으면,
            tfidf[token] = tf_value * idf[token]  # TF-IDF 값 계산
    return tfidf

# 여러 문서의 토큰 리스트 (예시)
# 여기서는 하나의 문서만 있으므로, 리스트 안에 넣어줌
documents_tokens = [tokens]  #

# TF 계산
tf = calculate_tf(tokens)  # 현재 문서의 TF 계산

# IDF 계산
idf = calculate_idf(documents_tokens)  # 전체 문서에 대한 IDF 계산

# TF-IDF 계산
tfidf = calculate_tfidf(tf, idf) # TF-IDF 값 계산
print("TF-IDF:\n", tfidf)

# TF-IDF 벡터로 변환 (순서 중요!)
def vectorize_tfidf(tfidf, word_index):
    """
    TF-IDF 딕셔너리를 벡터로 변환하는 함수.

    Args:
        tfidf: TF-IDF 딕셔너리.
        word_index: 단어-인덱스 매핑 딕셔너리 (전체 단어 집합에 대한).

    Returns:
        TF-IDF 벡터 (NumPy 배열).
    """
    vector = np.zeros(len(word_index))
    for token, value in tfidf.items():
        if token in word_index:
          index = word_index[token]
          vector[index] = value
    return vector

# 전체 단어 집합 (예시) - 실제로는 모든 문서에서 추출한 단어를 사용
all_tokens = set()
for doc_tokens in documents_tokens:
    all_tokens.update(doc_tokens)

# 단어-인덱스 매핑
word_index = {token: i for i, token in enumerate(all_tokens)}

# TF-IDF 벡터화
tfidf_vector = vectorize_tfidf(tfidf, word_index)
print("TF-IDF 벡터:\n", tfidf_vector)


## 4. 유사도 계산

**설명:**

1.  **`cosine_similarity` 함수:** 이전 답변에서 사용한 코사인 유사도 계산 함수를 재사용합니다.
2.  **다른 시놉시스 예시:** 비교를 위해 다른 시놉시스를 준비합니다.
3.  **전처리, 토큰화, TF-IDF 벡터화:** 두 번째 시놉시스에 대해서도 동일한 전처리, 토큰화, TF-IDF 벡터화 과정을 수행합니다.
    *   **중요:** IDF는 *두 시놉시스 모두를 포함하는 문서 집합*에 대해 계산해야 합니다.  그래야 두 벡터가 같은 어휘 공간(vocabulary space)을 기준으로 비교됩니다.
    *   `all_tokens.update(tokens2)`:  전체 단어 집합에 두번째 시놉시스 단어 추가.
    *   `word_index = {token: i for i, token in enumerate(all_tokens)}`: 단어-인덱스 매핑 업데이트
4.  **코사인 유사도 계산:** 두 TF-IDF 벡터 간의 코사인 유사도를 계산합니다.

이 코드는 두 시놉시스의 유사도를 계산하는 완전한 예시를 제공합니다. 불용어를 더 추가하거나, stemming/lemmatization을 적용하거나, 다른 유사도 측정 방법(예: 유클리드 거리)을 사용하는 등 필요에 따라 코드를 수정할 수 있습니다.


In [ ]:
from numpy.linalg import norm

def cosine_similarity(vector1, vector2):
    """
    두 벡터 간의 코사인 유사도를 계산하는 함수.

    Args:
        vector1: 첫 번째 벡터 (NumPy 배열).
        vector2: 두 번째 벡터 (NumPy 배열).

    Returns:
        코사인 유사도 (float).
    """
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = norm(vector1)
    norm_vector2 = norm(vector2)
    if norm_vector1 == 0 or norm_vector2 == 0: # 0으로 나누는 것 방지
      return 0
    else:
      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

# 다른 시놉시스 예시 (비교 대상)
synopsis2 = """
"선생님, 저랑 사귀실래요?" 적극적인 어른 민주
"꺼져" 철벽 많은 급식 윤서

윤서는 학교에서 학생들에게 인기가 매우 많은 선생님이다.
어느 날, 윤서는 민주로부터 고백을 받게 된다.
하지만 윤서는 민주를 거절한다.

윤서와 민주, 두 사람의 아슬아슬한 만남은 계속된다.
"""

# 전처리, 토큰화, TF-IDF 벡터화
preprocessed_synopsis2 = preprocess_synopsis(synopsis2)
tokens2 = tokenize(preprocessed_synopsis2)
documents_tokens2 = [tokens2] # 비교를 위해 문서 집합으로 변환
tf2 = calculate_tf(tokens2)

# IDF는 첫번째 시놉시스와 두번째 시놉시스를 모두 포함해서 계산
idf= calculate_idf(documents_tokens + documents_tokens2)

tfidf2 = calculate_tfidf(tf2, idf) # idf는 첫 번째 문서와 함께 계산한 것 사용

# 전체 단어 집합 갱신
all_tokens.update(tokens2)

# 단어-인덱스 매핑 업데이트
word_index = {token: i for i, token in enumerate(all_tokens)}

tfidf_vector2 = vectorize_tfidf(tfidf2, word_index)

# 코사인 유사도 계산
similarity = cosine_similarity(tfidf_vector, tfidf_vector2)
print("코사인 유사도:", similarity)